In [29]:
Method_number=7
#1:WebBert Similarity
#2:cosine similarity
#3:jaccard similarity
#4:bm25 similarity
#5:Sørensen-Dice coefficient
#6:Levenshtein Similarity
#7:Combination by above methods
#8:BLEU score
#9:rouge score
#10:iou
#11:Exact Match

output_file_name='extracted_data.xlsx'#change it to your file name, and don't forget to put it in the same folder with this program

In [3]:
!pip install spacy
!python -m spacy download en_core_web_sm
#package required for keywords extraction

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 80.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
!pip install semantic-text-similarity
!pip install Levenshtein
!pip install rank_bm25
!pip install nltk
!pip install rouge-score
#package required for similarity methods

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.0/416.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 18.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [5]:
from semantic_text_similarity.models import WebBertSimilarity
from semantic_text_similarity.models import ClinicalBertSimilarity

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from rank_bm25 import BM25Okapi
import jieba

import Levenshtein as lev

import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from rouge_score import rouge_scorer

In [6]:
import spacy

In [7]:
import re
import numpy as np

In [8]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def remove_before_colon(input_string):
    # Find the position of the first occurrence of ':'
    colon_pos = input_string.find(':')

    # If a colon is found, remove everything before it including the colon
    if colon_pos != -1:
        return input_string[colon_pos + 1:].strip()
    else:
        # If no colon is found, return the original string
        return input_string

In [10]:
def clean_string(input_string):
    # Replace newlines with a space to handle multi-line inputs as a single string
    input_string = input_string.replace('\n', ' ')

    # Remove leading numbers followed by a dot and optional whitespace
    input_string = re.sub(r'\d+\.\s*', '', input_string)

    # Remove all occurrences of * # - :
    input_string = re.sub(r'[\*#\-:]', '', input_string)

    # Remove extra spaces
    cleaned_string = ' '.join(input_string.split())

    return cleaned_string

In [11]:
# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

def extract_keywords(input_string):
    # Process the input string with spaCy NLP
    doc = nlp(input_string)
    # Extract named entities of type PERSON, TIME, and GPE (Geo-Political Entity)
    #keywords = [ent.text for ent in doc.ents if ent.label_ in {'PERSON', 'TIME', 'GPE', 'ORG'}]
    keywords = [ent.text for ent in doc.ents if ent.label_ in { 'PERSON','ORG'}]
    # Join the keywords with a comma and return
    return ', '.join(keywords)

In [12]:
def fill_empty_positions(input_list):
    # Iterate over the list and replace empty positions with 'Not Mentioned'
    return ['Not Mentioned' if not item else item for item in input_list]

In [ ]:
def extract_Offensekeywords(input_string):
    # Process the input string with spaCy NLP
    doc = nlp(input_string)
    # Extract named entities of type PERSON, TIME, and GPE (Geo-Political Entity)
    #keywords = [ent.text for ent in doc.ents if ent.label_ in {'PERSON', 'TIME', 'GPE', 'ORG'}]
    keywords = [ent.text for ent in doc.ents if ent.label_ in {'PERSON', 'ORG', 'GPE'}]
    # Join the keywords with a comma and return
    return ', '.join(keywords)

In [13]:
#method 2
def cos_similarity(sentence1,sentence2):
   # Vectorize sentences
   vectorizer = CountVectorizer().fit_transform([sentence1, sentence2])
   vectors = vectorizer.toarray()

   # Compute cosine similarity
   cosine_sim = cosine_similarity(vectors[0].reshape(1, -1), vectors[1].reshape(1, -1))[0][0]

   return cosine_sim

#method 3
def jaccard_similarity(sentence1, sentence2):
    set1 = set(sentence1.split())
    set2 = set(sentence2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

#method 4
def bm25_similarity(sentence1,sentence2):
    # Tokenize sentences
    tokenized_sentence1 = list(jieba.cut(sentence1))
    tokenized_sentence2 = list(jieba.cut(sentence2))
    #tokenized_sentence1 = sentence1.lower().split()
    #tokenized_sentence2 = sentence2.lower().split()

    # Initialize BM25 model
    bm25 = BM25Okapi([tokenized_sentence2])

    # Compute BM25 score
    bm25_score = bm25.get_scores(tokenized_sentence1)

    return bm25_score

#method 5
def sorensen_dice_coefficient(sentence1, sentence2):
    set1 = set(sentence1.split())
    set2 = set(sentence2.split())
    intersection = len(set1.intersection(set2))
    dice_coefficient = 2 * intersection / (len(set1) + len(set2))
    return dice_coefficient

#method 6
def lev_similarity(sentence1,sentence2):
   # Compute Levenshtein Distance
   lev_distance = lev.distance(sentence1, sentence2)
   # Normalize distance to a similarity score
   lev_similarity = 1 - lev_distance / max(len(sentence1), len(sentence2))

   return lev_similarity

#method 8
def calculate_bleu_score(reference_sentence, candidate_sentence):
    """
    Calculate the BLEU score between two sentences.

    Parameters:
    reference_sentence (str): The reference sentence.
    candidate_sentence (str): The candidate sentence.

    Returns:
    float: The BLEU score between the two sentences.
    """
    # Tokenize the sentences
    reference_tokens = [reference_sentence.split()]
    candidate_tokens = candidate_sentence.split()

    # Smoothing function to handle cases with small sample size
    smoothie = SmoothingFunction().method4

    # Calculate BLEU score
    bleu_score = sentence_bleu(reference_tokens, candidate_tokens, smoothing_function=smoothie)

    return bleu_score

#method 9
def calculate_rouge_score(reference_sentence, candidate_sentence):
    """
    Calculate the ROUGE-1 score between two sentences.

    Parameters:
    reference_sentence (str): The reference sentence.
    candidate_sentence (str): The candidate sentence.

    Returns:
    dict: The ROUGE-1 precision, recall, and F1 score.
    """
    # Initialize the ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

    # Calculate the ROUGE score
    scores = scorer.score(reference_sentence, candidate_sentence)

    return scores['rouge1']

#method 10
def calculate_iou(reference_sentence, candidate_sentence):
    """
    Calculate the Intersection over Union (IoU) score between two sentences.

    Parameters:
    reference_sentence (str): The reference sentence.
    candidate_sentence (str): The candidate sentence.

    Returns:
    float: The IoU score between the two sentences.
    """
    # Tokenize the sentences into sets of words
    reference_tokens = set(reference_sentence.split())
    candidate_tokens = set(candidate_sentence.split())

    # Handle edge case where both sentences are empty
    if not reference_tokens and not candidate_tokens:
        return 1.0

    # Handle edge case where one of the sentences is empty
    if not reference_tokens or not candidate_tokens:
        return 0.0

    # Calculate the intersection and union of the two sets
    intersection = reference_tokens.intersection(candidate_tokens)
    union = reference_tokens.union(candidate_tokens)

    # Calculate the IoU score
    iou_score = len(intersection) / len(union)

    return iou_score

#method 11
def normalize_text(text):
    """
    Normalize text by removing extra spaces, converting to lowercase, and stripping leading/trailing spaces.

    Parameters:
    text (str): The text to normalize.

    Returns:
    str: The normalized text.
    """
    text = text.strip().lower()
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

def calculate_exact_match(reference_sentence, candidate_sentence):
    """
    Calculate the Exact Match (EM) score between two sentences.

    Parameters:
    reference_sentence (str): The reference sentence.
    candidate_sentence (str): The candidate sentence.

    Returns:
    int: The EM score (1 if sentences match exactly, 0 otherwise).
    """
    # Normalize the sentences
    reference_sentence = normalize_text(reference_sentence)
    candidate_sentence = normalize_text(candidate_sentence)

    # Calculate EM score
    return 1 if reference_sentence == candidate_sentence else 0

In [14]:
label_data=pd.read_excel('drive/My Drive/Colab Notebooks/Spyglass165.xlsx')
#test_data=pd.read_excel('drive/My Drive/Colab Notebooks/extracted_data.xlsx')
test_data=pd.read_excel('drive/My Drive/Colab Notebooks/'+output_file_name)
#label_data=label_data.fillna('Not Mentioned'
#test_data['Vessel Name']
label_data=label_data.fillna('Not Mentioned')
test_data=test_data.fillna('Not Mentioned')
#label_data.iloc[16,21]

In [ ]:
#label_data['Crime Baskets for convergence']

In [15]:
len_compare=min(len(test_data),len(label_data))
compare_total=6
score_mat=np.zeros((len_compare,compare_total))

#different columns require different preprocessing since the structure of test_data and label_data has difference
#VesselName, VesselFlag, OffenseInformation and GoodsOnboard are easier since structure of them in both files are simple and similar(just names or items)
VesselName_cleaned=[]
VesselFlag_cleaned=[]
GoodsOnboard_cleaned=[]
Offense_cleaned=[]
Ownership_cleaned=[]
Arrestinfromation_cleaned=[]

VesselName_label=[]
VesselFlag_label=[]
GoodsOnboard_label=[]
Offense_label=[]
Ownership_label=[]
Arrestinfromation_label=[]
for i in range(len_compare):
  VesselName_cleaned.append(clean_string(test_data['Vessel Name'][i]))
  VesselFlag_cleaned.append(clean_string(test_data['Vessel Flag'][i]))
  GoodsOnboard_cleaned.append(clean_string(test_data['Goods Onboard'][i]))
  Offense_cleaned.append(clean_string(test_data['Offense information'][i]))
  Ownership_cleaned.append(clean_string(test_data['Ownership Information'][i]))
  Arrestinfromation_cleaned.append(clean_string(test_data['Arrest Information'][i]))

  VesselName_label.append(label_data['Vessel Name'][i])
  VesselFlag_label.append(label_data.iloc[i,8])
  GoodsOnboard_label.append(label_data.iloc[i,21])
  Offense_label.append(label_data['Offence information'][i])
  Ownership_label.append(label_data['Owner name'][i])
  Arrestinfromation_label.append(remove_before_colon(label_data['Crime Baskets for convergence'][i]+', arrested by '+label_data['Arresting authority'][i]))
#Arrestinfromation_cleaned

In [ ]:
#Ownership_cleaned

In [16]:
Ownership_keywords=[]
for i in range(len_compare):
  Ownership_keywords.append(extract_keywords(Ownership_cleaned[i]))

Ownership_keywords

['Sajo Oyang',
 '',
 'John Duncan, Jerry Ramsay',
 'ALS Fisheries',
 'Westfleet Fishing Limited']

In [ ]:
#Arrestinfromation_cleaned

In [ ]:
#Arrest_informationKeywords=[]
#for i in range(len_compare):
#  Arrest_informationKeywords.append(extract_Offensekeywords(Arrestinfromation_cleaned[i]))

#Arrest_informationKeywords

In [17]:
web_model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction
clinical_model = ClinicalBertSimilarity(device='cuda', batch_size=10) #defaults to GPU prediction

100%|██████████| 405359924/405359924 [00:08<00:00, 46657615.18B/s]


100%|██████████| 401555686/401555686 [00:06<00:00, 57494715.12B/s]


In [ ]:
#Arrestinfromation_cleaned

In [27]:
VesselName_cleaned=fill_empty_positions(VesselName_cleaned)
VesselFlag_cleaned=fill_empty_positions(VesselFlag_cleaned)
GoodsOnboard_cleaned=fill_empty_positions(GoodsOnboard_cleaned)
Offense_cleaned=fill_empty_positions(Offense_cleaned)
Ownership_cleaned=fill_empty_positions(Ownership_cleaned)
Arrestinfromation_cleaned=fill_empty_positions(Arrestinfromation_cleaned)
#unify empty contents that not mentioned

Method_number=7
if Method_number==1:
  for i in range(len_compare):
    score_mat[i,0]=web_model.predict([(VesselName_cleaned[i],VesselName_label[i])])
    score_mat[i,1]=web_model.predict([(VesselFlag_cleaned[i],VesselFlag_label[i])])
    score_mat[i,2]=web_model.predict([(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])])
    score_mat[i,3]=web_model.predict([(Offense_cleaned[i],Offense_label[i])])
    score_mat[i,4]=web_model.predict([(Ownership_keywords[i],Ownership_label[i])])
    score_mat[i,5]=web_model.predict([(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])])

elif Method_number==2:
  for i in range(len_compare):
    score_mat[i,0]=cos_similarity(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=cos_similarity(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=cos_similarity(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=cos_similarity(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=cos_similarity(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=cos_similarity(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

elif Method_number==3:
  for i in range(len_compare):
    score_mat[i,0]=jaccard_similarity(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=jaccard_similarity(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=jaccard_similarity(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=jaccard_similarity(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=jaccard_similarity(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=jaccard_similarity(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

elif Method_number==4:
  for i in range(len_compare):
    score_mat[i,0]=bm25_similarity(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=bm25_similarity(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=bm25_similarity(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=bm25_similarity(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=bm25_similarity(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=bm25_similarity(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

elif Method_number==5:
  for i in range(len_compare):
    score_mat[i,0]=sorensen_dice_coefficient(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=sorensen_dice_coefficient(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=sorensen_dice_coefficient(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=sorensen_dice_coefficient(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=sorensen_dice_coefficient(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=sorensen_dice_coefficient(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

elif Method_number==6:
  for i in range(len_compare):
    score_mat[i,0]=lev_similarity(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=lev_similarity(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=lev_similarity(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=lev_similarity(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=lev_similarity(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=lev_similarity(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

elif Method_number==7:#combine as you like
  for i in range(len_compare):
    score_mat[i,0]=calculate_exact_match(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=calculate_exact_match(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=calculate_iou(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=web_model.predict([(Offense_cleaned[i],Offense_label[i])])
    score_mat[i,4]=calculate_bleu_score(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=web_model.predict([(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])])

elif Method_number==8:
  for i in range(len_compare):
    score_mat[i,0]=calculate_bleu_score(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=calculate_bleu_score(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=calculate_bleu_score(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=calculate_bleu_score(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=calculate_bleu_score(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=calculate_bleu_score(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

elif Method_number==9:
  for i in range(len_compare):#choose precision
    score_mat[i,0]=calculate_rouge_score(VesselName_cleaned[i],VesselName_label[i])[0]
    score_mat[i,1]=calculate_rouge_score(VesselFlag_cleaned[i],VesselFlag_label[i])[0]
    score_mat[i,2]=calculate_rouge_score(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])[0]
    score_mat[i,3]=calculate_rouge_score(Offense_cleaned[i],Offense_label[i])[0]
    score_mat[i,4]=calculate_rouge_score(Ownership_keywords[i],Ownership_label[i])[0]
    score_mat[i,5]=calculate_rouge_score(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])[0]

elif Method_number==10:
  for i in range(len_compare):
    score_mat[i,0]=calculate_iou(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=calculate_iou(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=calculate_iou(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=calculate_iou(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=calculate_iou(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=calculate_iou(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

elif Method_number==11:
  for i in range(len_compare):
    score_mat[i,0]=calculate_exact_match(VesselName_cleaned[i],VesselName_label[i])
    score_mat[i,1]=calculate_exact_match(VesselFlag_cleaned[i],VesselFlag_label[i])
    score_mat[i,2]=calculate_exact_match(GoodsOnboard_cleaned[i],GoodsOnboard_label[i])
    score_mat[i,3]=calculate_exact_match(Offense_cleaned[i],Offense_label[i])
    score_mat[i,4]=calculate_exact_match(Ownership_keywords[i],Ownership_label[i])
    score_mat[i,5]=calculate_exact_match(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])

<ipython-input-27-7b029cea13a7>:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score_mat[i,3]=web_model.predict([(Offense_cleaned[i],Offense_label[i])])
<ipython-input-27-7b029cea13a7>:71: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score_mat[i,5]=web_model.predict([(Arrestinfromation_cleaned[i],Arrestinfromation_label[i])])


In [28]:
#score_mat
Final_score=sum(score_mat.mean(axis=1))/len_compare
Final_score

0.27896277819361004

# 新段落